In [ ]:
import pandas as pd
import numpy as np
import nsepy
from nsepy import get_history as web
from datetime import date
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')
from nsepy.derivatives import get_expiry_date

In [ ]:
fr = pd.read_csv('friday_pe_close.csv')
fr.set_index('Unnamed: 0',inplace=True)
fr.index = pd.to_datetime(fr.index)

In [ ]:
mon=pd.read_csv('monday_pe_open.csv')
mon.set_index('Unnamed: 0', inplace = True)
mon.index = pd.to_datetime(mon.index)
mon.index = mon.index - timedelta(days=3)

monpre = mon[['Open(+1000)','Open(+1050)','Open(+1100)','Open(+1150)','Open(+1200)','Open(+1250)','Open(+1300)','Open(+1350)','Open(+1400)','Open(+1450)','High(+1450)']]
monpre.rename(columns = {'Open(+1000)':'mon1000_open','Open(+1050)':'mon1050_open','Open(+1100)':'mon1100_open','Open(+1150)':'mon1150_open','Open(+1200)':'mon1200_open','Open(+1250)':'mon1250_open','Open(+1300)':'mon1300_open','Open(+1350)':'mon1350_open','Open(+1400)':'mon1400_open','Open(+1450)':'mon1450_open','High(+1450)':'Predicted_high1450'}, inplace = True)

In [ ]:
nf = pd.read_csv('nifty.csv')
nf.set_index('Date',inplace=True)
nf.index = pd.to_datetime(nf.index)

In [ ]:
nffr = nf[nf['day_of_week']=='Friday']
nffr_close = nffr[['Close']]
nffr_close.rename(columns = {'Close':'Nifty_Friday_close'},inplace=True)

In [ ]:
nffr_close

In [ ]:
nfmon= nf[nf['day_of_week']=='Monday']
nfmon_open = nfmon[['Open']]
nfmon_open.rename(columns={'Open':'Nifty_Monday_open'},inplace=True)
nfmon_open.index = nfmon_open.index - timedelta(days = 3)

In [ ]:
for_mon = pd.concat([fr,nffr_close,nfmon_open,monpre],axis = 1)
for_mon = for_mon.dropna()

for_mon

In [ ]:
X= for_mon.drop('Predicted_high1450',axis=1)
Y=for_mon['Predicted_high1450']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test=train_test_split(X,Y, test_size=0.4,random_state=30, shuffle=True)
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(x_train,y_train)

In [ ]:
print(lm.intercept_)

In [ ]:
coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
coeff_df

In [ ]:
predictions = lm.predict(x_test)
plt.scatter(y_test,predictions)

In [ ]:
df = pd.DataFrame()
df['Actuals'] = y_test
df['Pred'] = predictions
df

In [ ]:
df['MAPE'] = abs((df.Pred - df.Actuals) / df.Actuals) * 100
df['Accuracy'] = 100 - df.MAPE
df

In [ ]:
df.Accuracy.mean()